In [55]:
# 导入模块xml.etree中的ElementTree类，并取名为etree
from xml.etree import ElementTree as etree

In [56]:
from xml.etree.ElementTree import Element, SubElement, ElementTree

In [57]:
import pandas as pd

In [58]:
data=pd.read_csv(r'./test2.csv')

In [59]:
len(data)

9

In [60]:
# 生成根节点
kml = Element('kml')
for _ in range(len(data)):
    # 生成第一个子节点 document
    document = SubElement(kml, 'Document')

    # document 节点的子节点name
    placemark = SubElement(document, 'Placemark')

    name = SubElement(placemark,'name')
    name.text = '栅格'
    description = SubElement(placemark,'description')
    #description.text = 'RSRP:-75'

    #样式
    style = SubElement(placemark,'Style')

    polyStyle = SubElement(style,'PolyStyle')

    color = SubElement(polyStyle,'color')
    #color.text = 'ff00d500'

    outline = SubElement(polyStyle,'outline')
    outline.text = '0'

    #编辑属性Polygon
    polygon = SubElement(placemark,'Polygon')

    extrude = SubElement(polygon,'extrude')
    extrude.text = '1'

    altitudeMode = SubElement(polygon,'altitudeMode')
    altitudeMode.text = 'relativeToGround'

    outerBoundaryIs = SubElement(polygon,'outerBoundaryIs')
    linearRing = SubElement(outerBoundaryIs,'LinearRing')
    coordinates = SubElement(linearRing,'coordinates')
    #coordinates.text = '108.4702,22.91364,2.0 108.4702,22.91382,2.0 108.4704,22.91382,2.0 108.4704,22.91364,2.0'
tree = ElementTree(kml)

In [61]:
# elemnt为传进来的Elment类，参数indent用于缩进，newline用于换行   
def prettyXml(element, indent, newline, level = 0): 
    # 判断element是否有子元素
    if element:
        # 如果element的text没有内容      
        if element.text == None or element.text.isspace():     
            element.text = newline + indent * (level + 1)      
        else:    
            element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * (level + 1)    
    # 此处两行如果把注释去掉，Element的text也会另起一行 
    #else:     
        #element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * level    
    temp = list(element) # 将elemnt转成list    
    for subelement in temp:    
        # 如果不是list的最后一个元素，说明下一个行是同级别元素的起始，缩进应一致
        if temp.index(subelement) < (len(temp) - 1):     
            subelement.tail = newline + indent * (level + 1)    
        else:  # 如果是list的最后一个元素， 说明下一行是母元素的结束，缩进应该少一个    
            subelement.tail = newline + indent * level   
        # 对子元素进行递归操作 
        prettyXml(subelement, indent, newline, level = level + 1)     

In [65]:
from xml.etree import ElementTree      #导入ElementTree模块   

doc = tree.findall("Document")
for i in range(data.shape[0]):
    #print(i)
    tmp = doc[i]
    tmp.find('Placemark').find('description').text = 'RSRP:'+str(data.loc[i,'RSRPValue']) 
    #description.text = 'RSRP:'+str(data.loc[i,'RSRPValue']) 
    if data.loc[i,'RSRPValue']<-90:
        #color.text = 'ffFF1493'
        tmp.find('Placemark').find('Style').find('PolyStyle').find('color').text = 'ffFF1493'
    else:
        #color.text = 'ff00d500'
        tmp.find('Placemark').find('Style').find('PolyStyle').find('color').text = 'ff00d500'
    tmp.find('Placemark').find('Polygon').find('outerBoundaryIs').find('LinearRing').find('coordinates').text = str(data.loc[i,'TLLongitude']/10000000)+','+str(data.loc[i,'TLLatitude']/10000000)+','+str(2.0)+ ' '+str(data.loc[i,'TLLongitude']/10000000)+','+str(data.loc[i,'BRLatitude']/10000000)+','+str(2.0)+' '+str(data.loc[i,'BRLongitude']/10000000)+','+str(data.loc[i,'BRLatitude']/10000000)+','+str(2.0)+' '+str(data.loc[i,'BRLongitude']/10000000)+','+str(data.loc[i,'TLLatitude']/10000000)+','+str(2.0)
    
    
#检验是否可以采用data里面的数据给树的子节点赋值
#description.text ='RSRP:'+str(data.loc[3,'RSRPValue'])
#if data.loc[3,'RSRPValue']<-90:
    #color.text = 'ffFF1493'
#else:
   # color.text = 'ff00d500'
#coordinates.text = '108.4702,22.91364,2.0 108.4702,22.91382,2.0 108.4704,22.91382,2.0 108.4704,22.91364,2.0'   
    
    tree.write('resulttest.xml', encoding = 'utf-8')


    #tree = ElementTree.parse('result.xml')   #解析test.xml这个文件，该文件内容如上文    
    root = tree.getroot()                  #得到根元素，Element类    
    prettyXml(root, '\t', '\n')            #执行美化方法    

    #ElementTree.dump(root)                 #显示出美化后的XML内容  

    tree.write('maptest.kml', encoding = 'utf-8')